# 고객 이탈 예상 모델 생성 및 추론:

# [Module 1.0] 고객 이탈 데이타 수집 
---

## 비즈니스 배경 

고객을 잃은 다는 것은 어느 기업이나 손실이 큽니다. **불만이 있는 고객을 미리 인지한다는 것은, 미리 이탈 방지 할 수 있는 여러 프로모션을 줄 수 있는 기회를 제공** 합니다. 머신 러닝은 데이타를 통해서 이런 이탈 고객을 예상할 수 있는 방법을 제공 합니다. 물론 머신 러닝이 100% 정확한 예상을 할 수는 없지만, 이탈에 영향을 주는 의미 있는 데이타를 확보할 경우에는 예측 정확도는 상당히 올라 갑니다. 이런 모델을 통해서 고객 이탈 방지할 수 있는 가치를 제공 할 수 있습니다.

## 기술적 내용
본 워크샵에서는 아래와 같은 기술적인 요소를 포함 합니다.
- 피쳐 생성: SageMaker Processing 
    - 원본 데이타에 전처리 로직을 적용하여 전처리 데이타를 만드는 기술
- 모델 학습: SageMaker Built-in XGBoost Algorithm 사용
- Inference Pipeline: '전처리 --> 모델 추론 -> 후처리' 로 파이프라인 생성
- 배치 추론: SageMaker Batch Transform
- 실시간 추론: SageMaker Realtime Endpoint

- 이 워크샵은 아래 블로들을 참조하여 작성을 함.
- Reference:
    * Blog: Visualizing Amazon SageMaker machine learning predictions with Amazon QuickSight
        * https://aws.amazon.com/blogs/machine-learning/making-machine-learning-predictions-in-amazon-quicksight-and-amazon-sagemaker/
        * Git
            * https://github.com/aws-samples/quicksight-sagemaker-integration-blog
            
    * Blog: Preprocess input data before making predictions using Amazon SageMaker inference pipelines and Scikit-learn
        * https://aws.amazon.com/blogs/machine-learning/preprocess-input-data-before-making-predictions-using-amazon-sagemaker-inference-pipelines-and-scikit-learn/
        * Git: Inference Pipeline with Scikit-learn and Linear Learner
            * https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_inference_pipeline/Inference%20Pipeline%20with%20Scikit-learn%20and%20Linear%20Learner.ipynb

---
- 아래 노트북을 모두 실행을 하는데 약 2분 소요됨
---

## Setup Bucket and IAM Role
- 디폴트 버킷을 사용하고 현재 노트북 인스턴스의 Role을 가져옴. 사용자가 이미 생성한 버킷을 사용하고자 한다면 아래에 입력 하세요



In [1]:
# Set up Bucket
import sagemaker
bucket = sagemaker.Session().default_bucket()
# bucket = '<Your Bucket Name if you want to use it>'

prefix = 'sagemaker/customer-churn'

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import pandas as pd
import numpy as np
import os
from IPython.display import display

---
## Downlaod Data
- 미국의 임의의 이동 통신사 고객 이탈 데이타는 공개적으로 사용이 가능하고 아래에서 다운로드 가능합니다. 아래 언급된 책도 참조 가능 합니다.

    - The dataset we use is publicly available and was mentioned in the book [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) by Daniel T. Larose. It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.  Let's download and read that dataset in now:

In [3]:
# 사용할 폴더 지정
raw_data_folder = 'raw_data'
churn_data_folder = 'churn_data'

In [4]:
!wget --directory-prefix={raw_data_folder} http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o {raw_data_folder}/DKD2e_data_sets.zip -d {raw_data_folder}

--2020-07-18 06:27:15--  http://dataminingconsultant.com/DKD2e_data_sets.zip
Resolving dataminingconsultant.com (dataminingconsultant.com)... 160.153.91.162
Connecting to dataminingconsultant.com (dataminingconsultant.com)|160.153.91.162|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1003616 (980K) [application/zip]
Saving to: ‘raw_data/DKD2e_data_sets.zip.4’

DKD2e_data_sets.zip 100%[===================>] 980.09K  2.32MB/s    in 0.4s    

2020-07-18 06:27:15 (2.32 MB/s) - ‘raw_data/DKD2e_data_sets.zip.4’ saved [1003616/1003616]

Archive:  raw_data/DKD2e_data_sets.zip
 extracting: raw_data/Data sets/adult.zip  
  inflating: raw_data/Data sets/cars.txt  
  inflating: raw_data/Data sets/cars2.txt  
  inflating: raw_data/Data sets/cereals.CSV  
  inflating: raw_data/Data sets/churn.txt  
  inflating: raw_data/Data sets/ClassifyRisk  
  inflating: raw_data/Data sets/ClassifyRisk - Missing.txt  
 extracting: raw_data/Data sets/DKD2e data sets.zip  
  inflating: raw

### Read Churn Data

In [5]:
churn_file_name = os.path.join(raw_data_folder, 'Data sets', 'churn.txt')
churn = pd.read_csv(churn_file_name)
pd.set_option('display.max_columns', 500)
churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


미국의 어떤 이동 통신사의 총 3,333개의 레코드가 존재하고, 각 레코드는 21개의 컬럼이 존재 함.

- `State`: 미국의 주
- `Account Length`: 계정이 활성화 된 날짜의 수
- `Area Code`: 지역 번호
- `Phone`: 지역 번호 이외의 나머지 번호
- `Int’l Plan`: 고객이 해외 전화 연결 기능(international calling plan) 사용 여부. yes/no
- `VMail Plan`: 고객이 보이스 메일 기능 사용 여부: yes/no
- `VMail Message`: 월 평균 보이스 메일 평균 수
- `Day Mins`: 하루 동안 전화 통화 평균 분
- `Day Calls`: 하루 동안 전화 통화 평균 수
- `Day Charge`: 낮에 사용한 전화에 대한 청구 금액 
- `Eve Mins, Eve Calls, Eve Charge`: 저녁에 사용 (위의 낮의 정의 와 같음)
- `Night Mins`, `Night Calls`, `Night Charge`: 밤에 사용 (위의 낮의 정의와 같음)
- `Intl Mins`, `Intl Calls`, `Intl Charge`: 해외에 사용 (위의 낮의 정의와 같음)
- `CustServ Calls`: 고객 서비스 센터에 전화한 통화 수
- `Churn?`: 이 고객이 이탈할지 말지 여부. true/false (타켓 변수)


## 원본 데이타를 Train, Validation, Test 으로 분리
- Train을 70%, Validation 20%, Test 10%으로 분리 (각각, 2333, 666, 334 개 임)

In [6]:
os.makedirs(churn_data_folder, exist_ok=True)

# Set file names
train_file = os.path.join(churn_data_folder, 'train.csv')
validation_file = os.path.join(churn_data_folder, 'validation.csv')
test_batch_transform_test_file = os.path.join(churn_data_folder, 'batch_transform_test.csv')
test_batch_transform_test_true_file = os.path.join(churn_data_folder, 'batch_transform_test_true.csv')
test_file = os.path.join(churn_data_folder, 'test.csv')

In [7]:
train_data, validation_data, test_data = np.split(churn.sample(frac=1, random_state=1729), 
                                                  [int(0.7 * len(churn)), int(0.9 * len(churn))])
# 학습 파일
train_data.to_csv(train_file, header=False, index=False)
# 검증셋 파일
validation_data.to_csv(validation_file, header=False, index=False)

# 레이블을 포함한 테스트 파일 (추후 예측 결과와 비교를 위해 사용)
# As a ground truth, use for comparing prediction later
test_data.to_csv(test_batch_transform_test_true_file, header=False, index=False)
# 레이블을 제외한 테스트 파일
# Use it for batch_transform and realtime inference
test_data.drop('Churn?', axis=1).to_csv(test_batch_transform_test_file, header=False, index=False)

# For QuickSight
# test_data.drop('Churn?', axis=1).to_csv('test.csv', header=True, index=False)

In [8]:
print("train_data shape: ", train_data.shape)
print("validation_data shape: ", validation_data.shape)
print("test_data shape: ", test_data.shape)

train_data shape:  (2333, 21)
validation_data shape:  (666, 21)
test_data shape:  (334, 21)


## 세개의 파일을 S3에 업로드

In [9]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'rawtrain/train.csv')).upload_file(train_file)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'rawvalidation/validation.csv')).upload_file(validation_file)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'rawtest/batch_transform_test.csv')).upload_file(test_batch_transform_test_file)
# boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'rawtest/test.csv')).upload_file('test.csv')

S3 위치를 포인팅 하는 변수를 생성 함.

In [10]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/rawtrain/'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/rawvalidation/'.format(bucket, prefix), content_type='csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/rawtest/'.format(bucket, prefix), content_type='csv')

In [11]:
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/rawtrain/',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

In [12]:
s3_input_validation.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/rawvalidation/',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

In [13]:
s3_input_test.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/rawtest/',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

### 변수 저장하여 추후 노트북에서 사용

In [14]:
%store s3_input_train
%store s3_input_validation
%store s3_input_test
%store test_batch_transform_test_true_file
%store bucket
%store prefix

Stored 's3_input_train' (s3_input)
Stored 's3_input_validation' (s3_input)
Stored 's3_input_test' (s3_input)
Stored 'test_batch_transform_test_true_file' (str)
Stored 'bucket' (str)
Stored 'prefix' (str)
